# Zillow Webscraping
## Project description
- the goal the project
- pricing, sqft, lot size, etc.
- the whole bay area: counties
## Explore the dataset
-goals exploring dataset

## Price prediction


- Create seperate file to read into the url

- Define  list_or_url:
    clean up r, p, t
    make it readable
    
Move import statement outside as a global


## Packages


In [3]:
import requests
from bs4 import BeautifulSoup as BS
import json
import time
import csv
import os
import sys
import numpy as np
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

import fake_useragent
from fake_useragent import UserAgent

## Pulling URL from individual property card on Zillow
Keep in mind that html/json script is always changing. User will have to check if scripts are up to date.

In [9]:
class ZillowScraper():
    """
    This class here will use the header to by pass the zillow security features and parse through each of the
    property card and pull the URL from each of the cards and then save it to a csv file.
    ***Important***
    User will have to manually adjust the page range due to listing constantly being updated.
    Recommend looking at page first and then adjusting the range().
    """
    output = []
    headers = {
        
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'cookie': 'zguid=23|%248ce812ae-6c06-4282-a185-aa150258dbf1; zgsession=1|8334c6a7-229b-4e76-848b-82389e70947e; _ga=GA1.2.1328246037.1599671175; _gid=GA1.2.1914266924.1599671175; zjs_user_id=null; zjs_anonymous_id=%228ce812ae-6c06-4282-a185-aa150258dbf1%22; JSESSIONID=1708916C4FBDF470F11ECAC8FF6B9FE4; _gcl_au=1.1.1164169983.1599671175; KruxPixel=true; DoubleClickSession=true; _pxvid=c4cdb03c-f2be-11ea-8c78-0242ac120009; _fbp=fb.1.1599671175499.1034465933; _pin_unauth=dWlkPVpXVXpabUpsT1dZdE1URXlOeTAwTlRObExXSmpNbUV0TW1VM05qTTVPR0psWVRNeiZycD1abUZzYzJV; KruxAddition=true; __gads=ID=e626110456c836d9:T=1599671193:S=ALNI_Ma54r9zXdXhEGsTRKIfa-inIvF-fw; _uetsid=3f2d45905ff726b6499b8706116c0114; _uetvid=3559190482ab0e9b5caa1cac51f7c1a0; AWSALB=dMrLy2l7uELP5EH+AxX5MMqzhulxSiIOT4rBks/Jw84brvFEjZkNj5UbCeNRDfa3IFzxyG6ZOLgMyOhzt01WdfX2c7Vq0fTna+QWhxXFGphrS3FpESujWPIh9iuZ; AWSALBCORS=dMrLy2l7uELP5EH+AxX5MMqzhulxSiIOT4rBks/Jw84brvFEjZkNj5UbCeNRDfa3IFzxyG6ZOLgMyOhzt01WdfX2c7Vq0fTna+QWhxXFGphrS3FpESujWPIh9iuZ; search=6|1602264036452%7Crect%3D37.76954206618076%252C-121.40946584570312%252C36.84547153257956%252C-122.34055715429687%26rid%3D33839%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26type%3Dhouse%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0933839%09%09%09%09%09%09; _px3=7b4431112be2b28f4f7fa94f9d1a8de95193400e4c165b534dae141a82fac7d2:SYa7vqmqqOxn149LrL2d2vYCEiZZc6tT5Wdzvc2reryEePknRoBraFZnhzzHAgcxK+HcSZdbDYdBM1yOJvSa9g==:1000:gjXwxqSPFrWbs8JpZxS60goxu4ObF8+WiWxJdxryvNkuXmMvRANUuN8owbmRu3XZvQgZgh28PHhPvoR9gbdsln7BBiLMPgIW0LD+sdtC06kttkL/eEFdNP69vgnKHxokeyInMeXb0AyYzbBaHshBbN/g7SwnMXJa+7jm7hRZSgA=',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'

    }
    
    def fetch(self, URL, search_parameters):
      #Print request code
        web_page = requests.get(URL, headers = self.headers, params = search_parameters)
        print("Method Fetch Print:",web_page)

        return web_page
    
    def parse(self, web_page):
      #Turning webpage to soup and reading it as lxml
        content = BS(web_page, 'lxml')
        pages = content.find('ul',{'class': 'photo-cards photo-cards_wow photo-cards_short photo-cards_extra-attribution'})
      #For each card on Zillow site we are pulling the url of each one.
        for property in pages.contents:
            script = property.find('script', {'type': 'application/ld+json'})
            if script:
                script_json = json.loads(script.contents[0])
                self.output.append({
                    'url': script_json['url']         
                })

    def to_csv(self):
      #Creating a csv file
        with open('/Users/truon/Documents/BAN612/Zillow/GitTestRun/ZillowForSaleURLFeb12.csv', 'w') as csv_file:
            writer = csv.DictWriter(csv_file, lineterminator='\n', fieldnames=self.output[0].keys())
            writer.writeheader()
            for row in self.output:
                writer.writerow(row)    
    
    def run(self):
      #Main page of zillow with the city.
        URL = 'https://www.zillow.com/homes/San-Jose,-CA_rb/'
      #Pages on zillow that we will go through the range will have to be manually adjusted due to listing being updated.
      #If there are 16 pages of property card count ends at 15.
        for page in range(1,10):
            search_parameters = {
                'searchQueryState': '{"pagination":{"currentPage": %s},"mapBounds":{"west":-122.34055715429687,"east":-121.40946584570312,"south":36.84547153257956,"north":37.76954206618076},"regionSelection":[{"regionId":33839,"regionType":6}],"isMapVisible":True,"filterState":{"sort":{"value":"globalrelevanceex"},"con":{"value":False},"mf":{"value":False},"manu":{"value":False},"land":{"value":False},"tow":{"value":False},"apa":{"value":False}},"isListVisible":True}' %page
            }    
            res = self.fetch(URL,search_parameters)
            self.parse(res.text)
            time.sleep(1)
        self.to_csv()
            

if __name__ == '__main__':
    scraper = ZillowScraper()
    scraper.run()

Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>
Method Fetch Print: <Response [200]>


## Manually entering Proxies
    - Avoid Captcha and bans from zillow website.
    - Allow users to change their IP, socks, and proxy.
### How it works:
When driver variable is called it will open up the chromedriver extension. Fake useragent will then enter in the proxy, socks, and ssl the user agent has manually entered in.

In [25]:
prox = Proxy()
prox.proxy_type = ProxyType.MANUAL
prox.http_proxy = "85.175.227.3:7018" #Change this ip code if needed
#prox.socks_proxy = "ip_addr:port"
#prox.ssl_proxy = "ip_addr:port"
capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(chrome_options=options, executable_path=r'/Users/truon/Documents/BAN612/chromedriver', desired_capabilities=capabilities)
#https://chromedriver.chromium.org/downloads
#Make sure you have the right Chrome Version on your computer.

<ipython-input-25-abba95e2793f>:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'/Users/truon/Documents/BAN612/chromedriver', desired_capabilities=capabilities)


In [26]:
def get_property_detail(property_url_list, driver, start_index, end_index):
    Price_per_sqft = []
    status = []
    listPrice = []
    Type = []
    Year = []
    Heating = []
    Cooling = []
    Parking = []
    Space =[]
    HOA = []
    Lot = []
    daysOnZillow = []
    UrlList = []
    fullAddress =[]
    streetList = []
    cityList = []
    stateList =[]
    zipCodeList = []
    num_bed= []
    num_bath= []
    sqft_list = []
       
    driver.get(str(property_url_list[start_index]))
    soup = BS(driver.page_source, 'html.parser')  
    for i in range(start_index,end_index):
        #check status
        try:
            status_soup=soup.find_all("span", class_="qf5kuj-5 bkSUKT ds-status-details")
            if len(status_soup) == 0:
                status_soup=soup.find_all("span", class_="sc-pscky hEQFyS ds-status-details")
            if 'for sale' in status_soup[0].text.lower():
                status.append('For Sale')
            else:
                status.append(None)
        except:
            status.append(None)
        prop_soup=soup.findAll("span", class_="ds-bed-bath-living-area")
        time.sleep(np.random.randint(5,10))
    #Bed
        try:
            bed = prop_soup[0].text.replace(' bd','')
            num_bed.append(int(bed))
        except:
            num_bed.append(None)
        time.sleep(np.random.randint(1,2))
    #Bath   
        try:
            bath = prop_soup[1].text.replace(' ba','')
            num_bath.append(float(bath))
        except:
            num_bath.append(None)
            
        time.sleep(np.random.randint(1,2))
    #Sqft
        try:
            sqft = prop_soup[2].text.replace(' sqft','').replace(',','')
            sqft_list.append(int(sqft))
        except:
            sqft_list.append(None)
        time.sleep(np.random.randint(1,2))
        
        
#FACT AND FEATURE LABEL
        fact_label_soup=soup.find_all("div", class_="ds-expandable-card-section-default-padding")
        fact_label_list=[fact_label_soup[i].text for i in range(len(fact_label_soup))]
        fact_soup=soup.find_all("span", class_="ds-home-fact-list-item")
#House Type
        try:
            if 'Type:' in fact_label_list:
                Type.append(fact_soup[fact_label_list.index('Type:')].text)
            else:
                Type.append(None)
        except:
            Type.append(None)
        time.sleep(np.random.randint(1,2))
#Year built
        try:
            if 'Year built:' in fact_label_list:
                Year.append(fact_soup[fact_label_list.index('Year built:')].text)
            else:
                Year.append(None)
        except:
            Year.append(None)
        time.sleep(np.random.randint(1,3))
#Heating
        try:
            if 'Heating:' in fact_label_list:
                Heating.append(fact_soup[fact_label_list.index('Heating:')].text)
            else:
                Heating.append(None)
        except:
            Heating.append(None)
        time.sleep(np.random.randint(1,2))
#Cooling
        try:
            if 'Cooling:' in fact_label_list:
                Cooling.append(fact_soup[fact_label_list.index('Cooling:')].text)
            else:
                Cooling.append(None)
        except:
            Cooling.append(None)
        time.sleep(np.random.randint(1,3))
#Parking
        try:
            if 'Parking:' in fact_label_list:
                text=fact_soup[fact_label_list.index('Parking:')].text.lower()
                if text.startswith('no data')==True:
                    Parking.append(None)
                    Space.append(None)
                elif 'space' in text:
                    Parking.append('Space')
                    Space.append(int(re.findall('(.+?) spaces?',text)[0]))
                elif text.startswith('attached garage')==True:
                    Parking.append("Attached Garage")
                    Space.append('NA')
                elif text.startswith('carport')==True:
                    Parking.append('Carport')
                    Space.append('NA')
                else:
                    Parking.append(text)
                    Space.append('NA')
            else:
                Parking.append('No Parking')
                Space.append(0)
        except:
            Parking.append(None)
            Space.append(None)
        time.sleep(np.random.randint(1,2))
#HOA
        try:
            if 'HOA:' in fact_label_list:
                text=fact_soup[fact_label_list.index('HOA:')].text.lower()
                if '/month' in text:
                    HOA.append(int(re.findall('\$(.+?)/month',text)[0].replace(',','')))
                elif 'none' in text:
                    HOA.append(0)
                else:
                    HOA.append(text)
            else:
                HOA.append(None)
        except:
            HOA.append(None)
        time.sleep(np.random.randint(1,3))
#Price/sqft
        try:
            if 'Price/sqft:' in fact_label_list:
                text=fact_soup[fact_label_list.index('Price/sqft:')].text.lower()
                Price_per_sqft.append(int(text.replace('$','').replace(',','')))
            else:
                Price_per_sqft.append(None)
        except:
            Price_per_sqft.append(None)
        time.sleep(np.random.randint(1,2))
#Lot
        try:
            if 'Lot:' in fact_label_list:
                text=fact_soup[fact_label_list.index('Lot:')].text.lower()
                if 'acres' in text:
                    Lot.append(round(float(re.findall('(.*?) acres',text)[0])*43560,0))
                elif 'sqft' in text:
                    sqft=text.replace(',','').replace(' sqft','')
                    Lot.append(int(sqft))
                else:
                    Lot.append(text)
            else:
                Lot.append(None)
        except:
            Lot.append(None)
        time.sleep(np.random.randint(1,2))
            
#Days On Zillow
        days = soup.find('div', class_="sc-qamJO fiTUCr")
        try:
            if "days" in days.text:
                daysOnZillow.append(days.text)
            elif 'hours' in days.text:
                daysOnZillow.append(days.text)
            else:
                daysOnZillow.append(None)
        except:
            daysOnZillow.append(None)
        time.sleep(np.random.randint(1,3))
#URL
        photoCard = soup.find('div', class_= "ds-data-col ds-white-bg ds-data-col-data-forward")
        link = photoCard.find('script', {'type': 'application/ld+json'})
        script_json = json.loads(link.contents[0])
        urlTail = str(script_json['url'])
        fullLink = 'https://www.zillow.com'+urlTail
        UrlList.append(fullLink)
        time.sleep(np.random.randint(1,2))
#Listing price
        priceSoup = soup.find('h4', class_="Text-c11n-8-18-0__aiai24-0 StyledHeading-c11n-8-18-0__ktujwe-0 gcaUyc sc-qZtCU kijwKF")
        try:
            if "$" in priceSoup.text:
                cleanPrice = priceSoup.text.replace('$','').replace(',','')
                listPrice.append(int(cleanPrice))
            else:
                listPrice.append(None)
        except:
            listPrice.append(None)
        time.sleep(np.random.randint(1,2))


#Address
        address = soup.find('h1', class_='Text-c11n-8-18-0__aiai24-0 StyledHeading-c11n-8-18-0__ktujwe-0 efSAZl')
        cleanAddress = address.text.replace('\xa0','')
        splitAddress= cleanAddress.split(',')
        street = splitAddress[0]
        city = splitAddress[1]
        stateZip = splitAddress[2].split(' ')
        state = stateZip[1]
        zipCode = stateZip[2]
        
        
        streetList.append(street)
        cityList.append(city)
        stateList.append(state)
        zipCodeList.append(zipCode)
        fullAddress.append(cleanAddress)


        time.sleep(np.random.randint(15,20))
        driver.delete_all_cookies() 
        driver.get(str(property_url_list[i+1]))
        soup = BS(driver.page_source, 'html.parser')
    return status, num_bed, num_bath, sqft_list, listPrice, Type, Year, Heating, Cooling, Parking, Space, HOA, Price_per_sqft, Lot, daysOnZillow, UrlList, streetList, cityList, stateList, zipCodeList, fullAddress


def main(csv,start_index, end_index):
    url_df=pd.read_csv(csv)
    scrape_data=get_property_detail(url_df['url'], driver, start_index, end_index)
    df_output=url_df.iloc[start_index:end_index]
    df_output["Status"]=scrape_data[0]
    df_output["Num of bed"]=scrape_data[1]
    df_output["Num of bath"]=scrape_data[2]
    df_output["Sqft"]=scrape_data[3]
    df_output["Listing Price"]=scrape_data[4]
    df_output["House Type"]=scrape_data[5]
    df_output["Year built"]=scrape_data[6]
    df_output["Heating"]=scrape_data[7]
    df_output["Cooling"]=scrape_data[8]
    df_output["Parking Type"]=scrape_data[9]
    df_output["Num of parking space"]=scrape_data[10]
    df_output["HOA"]=scrape_data[11]
    df_output["Price per sqft"]=scrape_data[12]
    df_output["Lot Size in sqft"]=scrape_data[13]
    df_output["Days On Zillow"]=scrape_data[14]
    df_output["Property Url"]=scrape_data[15]
    df_output["Street"]=scrape_data[16]
    df_output["City"]=scrape_data[17]
    df_output["State"]=scrape_data[18]
    df_output["Zip Code"]=scrape_data[19]
    df_output["Full Address"]=scrape_data[20]
    return df_output

#Will index through by 10
ForSaleDF = main('/Users/truon/Documents/BAN612/Zillow/GitTestRun/ZillowForSaleURLFeb12.csv',0,5)
ForSaleDF.to_csv('/Users/truon/Documents/BAN612/Zillow/GitTestRun/SJFeb12_sale.csv', index=False)
#ForSaleDF = main('/Users/truon/Documents/BAN612/Zillow/ZillowSoldURLSanMateo.csv',0,10)
#ForSaleDF.to_csv('/Users/truon/Documents/BAN612/Zillow/SanMateoForSale.csv', index=False) #index=False, mode='a', header=False) to append into csv
#Scroll down for the header
ForSaleDF.head()

<ipython-input-26-aff2aaf8ccdd>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Status"]=scrape_data[0]
<ipython-input-26-aff2aaf8ccdd>:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Num of bed"]=scrape_data[1]
<ipython-input-26-aff2aaf8ccdd>:236: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,url,Status,Num of bed,Num of bath,Sqft,Listing Price,House Type,Year built,Heating,Cooling,...,HOA,Price per sqft,Lot Size in sqft,Days On Zillow,Property Url,Street,City,State,Zip Code,Full Address
0,https://www.zillow.com/homedetails/0-Cherrysto...,None,4,3.0,3400,900000,None,None,None,None,...,None,None,None,Time on Zillow189 days,https://www.zillow.com/homedetails/0-Cherrysto...,0 Cherrystone Dr,San Jose,CA,95128,"0 Cherrystone Dr,San Jose, CA 95128"
1,https://www.zillow.com/homedetails/1715-Hogar-...,None,3,2.0,1288,998999,None,None,None,None,...,None,None,None,Time on Zillow184 days,https://www.zillow.com/homedetails/1715-Hogar-...,1715 Hogar Dr,San Jose,CA,95124,"1715 Hogar Dr,San Jose, CA 95124"
2,https://www.zillow.com/homedetails/476-Arleta-...,None,3,2.0,1196,899888,None,None,None,None,...,None,None,None,Time on Zillow6 days,https://www.zillow.com/homedetails/476-Arleta-...,476 Arleta Ave,San Jose,CA,95128,"476 Arleta Ave,San Jose, CA 95128"
3,https://www.zillow.com/homedetails/393-Springp...,None,4,3.0,2075,1098000,None,None,None,None,...,None,None,None,Time on Zillow4 days,https://www.zillow.com/homedetails/393-Springp...,393 Springpark Cir,San Jose,CA,95136,"393 Springpark Cir,San Jose, CA 95136"
4,https://www.zillow.com/homedetails/5266-Vera-L...,None,3,3.0,1874,799000,None,None,None,None,...,None,None,None,Time on Zillow12 days,https://www.zillow.com/homedetails/5266-Vera-L...,5266 Vera Ln,San Jose,CA,95111,"5266 Vera Ln,San Jose, CA 95111"


In [ ]:
zillow_df=pd.DataFrame(columns=["Url","Address","Status","Bed","Bath","Price","Year built","Days on zillow","Lot size(sqft)","House size(sqft)","Heating","Cooling","Parking type","Parking space","Price/sqft","HOA"])
City=['SanJose','Richmond','SanLeandro','SanMateo']

#For Sale
for i in City:
    df=pd.DataFrame()
    print('processing',i+"'s sale data")
    file_h=pd.read_csv((i+'_sale.csv'))
    df['Url']=file_h['Url']
    df['Address']=file_h['Address']
    df.insert(1, "City", i,  allow_duplicates=True)
    df['Status']='Sale'
    df['Bed']=[str(i).replace(' bd','') for i in file_h['Bed']]
    df['Bath']=[str(i).replace(' ba','') for i in file_h['Bath']]
    Price_list=[]
    index=0
    for i in file_h["Price"]:
        i=str(i)
        if i=='None' or i==None:
            print('row',str(index),'is missing Price')
            Price_list.append(None)
        else:
            if 'm' in i.lower():
                new_p=float(i.lower().replace('m','').replace('$',''))*1000000
                Price_list.append(new_p)
            else:
                Price_list.append(float(i.replace(',','').replace('$','')))
        #print(f'price index:%d' %index)
        index+=1
    df['Price']=Price_list
    Year_list=[i if (i !='None') or (i !=None) else None for i in file_h['Year built']]
    df['Year built']=Year_list
    Days_zillow=[]
    for i in file_h['Days on zillow']:
        i=str(i)
        if i=='None' or i==None:
            day=None
        elif 'day' in i.lower():
            day=float(i.lower().replace(' day','').replace('s',''))
        elif 'hour' in i.lower():
            day=float(i.lower().replace(' hour','').replace('s',''))/24
        else:
            day=float(i)
        Days_zillow.append(day)
    df['Days on zillow']=Days_zillow
    Lot_size=[]
    for i in file_h['Lot size(sqft)']:
        i=str(i)
        if i=='None' or i==None or i.lower()=='no data':
            lot_sqft=None
        elif 'acres' in i.lower():
            lot_sqft=round(float(re.findall('(.*?) acres',i.lower())[0])*43560,0)
        elif 'sqft' in i.lower():
            lot_sqft=float(i.lower().replace(',','').replace(' sqft',''))
        else:
            lot_sqft=float(i)
        Lot_size.append(lot_sqft)
    df['Lot size(sqft)']=Lot_size
    House_size=[]
    for i in file_h["House size(sqft)"]:
        i=str(i)
        if i=='None' or i==None or i.lower()=='no data':
            house=None
        elif 'sqft' in i.lower():
            house=float(i.lower().replace(',','').replace(' sqft',''))
        elif 'square feet' in i.lower():
            house=float(i.lower().replace(',','').replace(' square feet',''))
        else:
            house=float(i)
        House_size.append(house)
    df["House size(sqft)"]=House_size
    #heat_list=[]
    #for i in file_h["Heating"]:
     #   if i=None or i.lower()='no data':
      #      heat=None
       # else:
        #    heat=i
        #heat_list.append(heat)
    df["Heating"]=[None if (i==None) or (str(i).lower()=='no data') else str(i) for i in file_h["Heating"]]
    df["Cooling"]=[None if (i==None) or (str(i).lower()=='no data') else str(i) for i in file_h["Cooling"]]
    Parking_type=[]
    Space_num=[]
    if 'Parking' in file_h.columns:
        for i in file_h['Parking']:
            i=str(i)
            if i==None:
                Parking_type.append(None)
                Space_num.append(None)
            else:
                text=i.lower()
                if 'no data' in text:
                    Parking_type.append(None)
                    Space_num.append(None)
                elif 'space' in text:
                    Parking_type.append('Space')
                    Space_num.append(int(re.findall('(.+?) spaces?',text)[0]))
                elif 'garage' in text:
                    Parking_type.append("Garage")
                    Space_num.append('NA')
                elif 'carport' in text:
                    Parking_type.append('Carport')
                    Space_num.append('NA')    
                elif 'off street' in text:
                    Parking_type.append('Off Street')
                    Space_num.append('NA') 
                else:
                    Parking_type.append('No Parking')
                    Space_num.append(0)
        df["Parking type"]=Parking_type
        df["Parking space"]=Space_num
    else:
        df["Parking space"]=file_h["Parking space"]
        for i in file_h['Parking type']:
            i=str(i)
            if i==None:
                Parking_type.append(None)
            else:
                text=i.lower()
                if 'no data' in text:
                    Parking_type.append(None)
                elif 'space' in text:
                    Parking_type.append('Space')
                elif 'garage' in text:
                    Parking_type.append("Garage")
                elif 'carport' in text:
                    Parking_type.append('Carport')
                elif 'off street' in text:
                    Parking_type.append('Off Street')
                else:
                    Parking_type.append('No Parking')
        df["Parking type"]=file_h["Parking type"]
    price_sqft=[None if (i==None) or (str(i).lower()=='none') or (str(i).lower()=='no data') else float(str(i).replace('$','').replace(',','')) for i in file_h["Price/sqft"]]
    df["Price/sqft"]=price_sqft
    df['HOA']=[None if (str(i)=='nan') or (i==None) else 0 if str(i).lower()=='none' else str(i).lower().replace('$','').replace('/month','') for i in file_h["HOA"]]
    zillow_df=zillow_df.append(df)
zillow_df.insert(15, "Sold date", "99/99/9999") 
zillow_df.head()

# Data Cleaning

In [ ]:
dfv3=pd.read_csv("dfv3_test.csv")
dfv3=dfv3.drop(columns=['Unnamed: 0'])
#filter out Price, Bed and Bath na
dfv3=dfv3[dfv3["Bed"].isna()==False]
dfv3=dfv3[dfv3["Bath"].isna()==False]
dfv3=dfv3[dfv3["Price"].isna()==False]
dfv3.isna().sum()

Filled missing value for Years with the mode based on the city.

In [27]:
mode=dfv3[dfv3["City"]=='SanJose']['Year built'].mode()[0]
print(mode)
dfv3['Year built'].fillna(mode,inplace = True)
dfv3['Year built'].isna().sum()

NameError: name 'dfv3' is not defined

Filled in missing value of lot size with median

In [ ]:
#Lot size(sqft):median
median=dfv3['Lot size(sqft)'].median()
print(median)
dfv3['Lot size(sqft)'].fillna(median,inplace = True)
dfv3['Lot size(sqft)'].isna().sum()

In [ ]:
Filled in missing value for Heating with mode and then transform into a Boolean